# Naural language Processing
## Word Embeddings for sentence sematics


In [4]:
import tensorflow as tf
print(tf.__version__)

2.3.0


### Importing the Data from Tensorflow data service

In [5]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised= True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSCK4ZO/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSCK4ZO/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteSCK4ZO/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

### Preprocessing the Data for model Training and Testing

In [7]:
import numpy as np

train_data,  test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []

for s,l in train_data:
    training_sentences.append(s.numpy().decode('utf-8'))
    training_labels.append(l.numpy())

for s,l in test_data:
    testing_sentences.append(s.numpy().decode('utf-8'))
    testing_labels.append(l.numpy())

training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

### Preprocessing the Text in the Data

In [19]:
vocab_size = 10000
max_len = 120
embedding_dim = 16
trunc_type = 'post'
oov_tok = '<OOV>'


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, truncating = trunc_type, maxlen = max_len)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen = max_len)

#### Reversing the word index dictionary:

In [20]:
reversed_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [21]:
def decode_review(text):
    return " ".join([reversed_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [28]:
model = tf.keras.Sequential([
                                    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_len),
                                    tf.keras.layers.Flatten(),
                             #tf.keras.layers.GlobalAveragePooling1D(),
                                    tf.keras.layers.Dense(6, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [29]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_3 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [30]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs= num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 3s 4ms/step - loss: 0.4875 - accuracy: 0.7486 - val_loss: 0.3447 - val_accuracy: 0.8485
Epoch 2/10
782/782 [==============================] - 3s 4ms/step - loss: 0.2391 - accuracy: 0.9067 - val_loss: 0.3739 - val_accuracy: 0.8377
Epoch 3/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0971 - accuracy: 0.9757 - val_loss: 0.4512 - val_accuracy: 0.8298
Epoch 4/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0242 - accuracy: 0.9964 - val_loss: 0.5467 - val_accuracy: 0.8246
Epoch 5/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0058 - accuracy: 0.9997 - val_loss: 0.6027 - val_accuracy: 0.8273
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.6514 - val_accuracy: 0.8282
Epoch 7/10
782/782 [==============================] - 3s 4ms/step - loss: 8.4038e-04 - accuracy: 1.0000 - val_loss: 0.6946 - val_accuracy: 0.8278
Ep

In [31]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


## For visualizing our Word Embeddings

In [32]:
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding = 'utf-8')

for word_num in range(1, vocab_size):
    word = reversed_word_index[word_num]
    embeddings = weights[word_num]

    out_m.write(word+ '\n')
    out_v.write('\t'.join([str(x) for x in embeddings]) + '\n')
out_v.close()
out_m.close()

In [33]:
try:
    from google.colab import files
except ImportError:
    pass
else:
    files.download('vecs.tsv')
    files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
sentence = 'I really think this is amazing, honest.'
sequence = tokenizer.texts_to_sequences(sentence)
print(sequence)

[[11], [], [1478], [960], [4], [2008], [2008], [5123], [], [828], [2021], [11], [3355], [2290], [], [828], [2021], [11], [588], [], [11], [588], [], [4], [1978], [4], [4555], [11], [3355], [1329], [], [], [2021], [1602], [3355], [960], [588], [828], []]


## View your Data visualization Here:  https://projector.tensorflow.org/